In [1]:
import numpy as np
import random

# Define the distance matrix (example with 4 cities)
distance_matrix = np.array([
    [0, 10, 15, 20],
    [10, 0, 35, 25],
    [15, 35, 0, 30],
    [20, 25, 30, 0]
])

# Parameters for Ant Colony Optimization
num_ants = 10
num_iterations = 100
evaporation_rate = 0.5
pheromone_constant = 1.0
heuristic_constant = 2.0

# Initialize pheromone matrix and visibility matrix
num_cities = len(distance_matrix)
pheromone = np.ones((num_cities, num_cities))  # Initial pheromone matrix
visibility = 1 / (distance_matrix + np.eye(num_cities)*1e-10)  # Avoid division by zero

# ACO algorithm
best_route = None
best_distance = float('inf')

for iteration in range(num_iterations):
    ant_routes = []
    
    for ant in range(num_ants):
        current_city = random.randint(0, num_cities - 1)
        visited_cities = [current_city]
        route = [current_city]

        while len(visited_cities) < num_cities:
            unvisited = [city for city in range(num_cities) if city not in visited_cities]
            probabilities = []
            
            for city in unvisited:
                pheromone_value = pheromone[current_city][city]
                visibility_value = visibility[current_city][city]
                probability = (pheromone_value ** pheromone_constant) * (visibility_value ** heuristic_constant)
                probabilities.append((city, probability))
            
            # Normalize probabilities
            total = sum(p[1] for p in probabilities)
            probabilities = [(p[0], p[1]/total) for p in probabilities]
            
            # Select next city probabilistically
            selected_city = random.choices(
                [p[0] for p in probabilities],
                weights=[p[1] for p in probabilities]
            )[0]
            
            route.append(selected_city)
            visited_cities.append(selected_city)
            current_city = selected_city

        ant_routes.append(route)

    # Update pheromone levels
    delta_pheromone = np.zeros((num_cities, num_cities))
    
    for route in ant_routes:
        route_distance = sum(distance_matrix[route[i]][route[(i+1)%num_cities]] for i in range(num_cities))
        
        for i in range(num_cities):
            city_i = route[i]
            city_j = route[(i+1)%num_cities]
            delta_pheromone[city_i][city_j] += 1 / route_distance
            delta_pheromone[city_j][city_i] += 1 / route_distance

    pheromone = (1 - evaporation_rate) * pheromone + delta_pheromone

    # Update best route
    for route in ant_routes:
        current_distance = sum(distance_matrix[route[i]][route[(i+1)%num_cities]] for i in range(num_cities))
        if current_distance < best_distance:
            best_distance = current_distance
            best_route = route

print("Best route:", best_route)
print("Shortest distance:", best_distance)

Best route: [0, 2, 3, 1]
Shortest distance: 80
